In [ ]:
import dataset

from dataset import FusedSentenceMichigan



DATASET =FusedSentenceMichigan(feature_options={},dataset_size=500)



In [ ]:
from dataset import run_segmentation

SEGMENTATION_RESULTS, DATASET_PROPERTIES = run_segmentation(dataset=DATASET)

In [ ]:
DATASET_PROPERTIES

In [ ]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import processes_segementation_results_global
PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET = processes_segementation_results_global(SEGMENTATION_RESULTS)

In [ ]:
PROCESSED_SEGEMENTATION_DATASET

In [ ]:
import pandas as pd
def convert_segmentations_to_index(segmentations, convert_fn = lambda x: x.replace("5", "4")):
    finalResults = []
    for i in range(len(segmentations)):
        results_entry = segmentations.iloc[i]
        word_count = results_entry['word_count']
        sentence = results_entry['sentence_results']
        mappings = results_entry['mappings']
        split_sentence = sentence.split("_")
        split_sentence = [convert_fn(x) for x in split_sentence]
        for j in range(len(split_sentence)):
            word_id = split_sentence[j] 
            toneclass = int(word_id[-1])
            main_Idx = i
            word_Idx = j
            mapped_onset = mappings[j]
            finalResults.append((word_id, toneclass, main_Idx, word_Idx,mapped_onset))

        
    return pd.DataFrame(finalResults, columns=['word_id', 'toneclass', 'main_Idx', 'word_Idx', 'mapped_onset'])




PSEG_INDEX = convert_segmentations_to_index(PROCESSED_SEGEMENTATION_DATASET)
PSEG_INDEX

In [ ]:
import librosa
def get_audio_sample_at_idx(idx,pSEG_index, pSEG, pSEGAUDIO, sr= 16000):
    sample_info = pSEG_index.iloc[idx]
    pSEG_idx = sample_info['main_Idx']

    sentence_info = pSEG.iloc[pSEG_idx]
    pSEG_audio = pSEGAUDIO.iloc[pSEG_idx]


    segementation_start_time_gt = sentence_info['delimiter_time_results']
    segementation_start_time = sentence_info['segementation_times_results'] + [segementation_start_time_gt[-1]]
    segmentation_onset_mapping = sentence_info['mappings']
    mapped_onset = sample_info['mapped_onset']

    current_mapping = segmentation_onset_mapping[sample_info['word_Idx']]

    if current_mapping == -1:
        # unmapped, use previous result
        prev_mapping = segmentation_onset_mapping[sample_info['word_Idx'] - 1]
        start_time = segementation_start_time[prev_mapping]
        end_time = segementation_start_time[prev_mapping+1]    
    else:
        start_time = segementation_start_time[current_mapping]
        end_time = segementation_start_time[current_mapping+1]

    start_samples = librosa.time_to_samples(start_time, sr=sr)
    end_samples = librosa.time_to_samples(end_time, sr=sr)

    return pSEG_audio[start_samples:end_samples]

for i in range(len(PSEG_INDEX)):
    print(get_audio_sample_at_idx(i, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET))